In [ ]:
# daily_fetch.py
import requests, json
import pandas as pd
from datetime import datetime
import os

HEADERS = {"User-Agent": "Mozilla/5.0"}
API_URL = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"

def fetch_data():
    sess = requests.Session()
    sess.get("https://www.nseindia.com", headers=HEADERS)
    resp = sess.get(API_URL, headers=HEADERS, cookies=sess.cookies)
    return resp.json()

def save_daily_data():
    today = datetime.today().strftime('%Y-%m-%d')
    json_data = fetch_data()
    expiry = json_data['records']['expiryDates'][0]
    rows = []

    for item in json_data['records']['data']:
        if item['expiryDate'] != expiry or 'CE' not in item:
            continue
        ce = item['CE']
        rows.append({
            'date': today,
            'strikePrice': ce['strikePrice'],
            'expiry': ce['expiryDate'],
            'OI': ce['openInterest'],
            'changeInOI': ce['changeinOpenInterest'],
            'volume': ce['volume'],
            'LTP': ce['lastPrice']
        })

    df = pd.DataFrame(rows)

    os.makedirs("data", exist_ok=True)
    path = f"data/nifty_options_chain.csv"

    if os.path.exists(path):
        df.to_csv(path, mode='a', index=False, header=False)
    else:
        df.to_csv(path, index=False)
    print(f"✅ Data saved for {today}")

if __name__ == "__main__":
    save_daily_data()
